<a href="https://colab.research.google.com/github/SantiM01/Senior-Project/blob/main/Game_by_Game_rush_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('filtered_by_player_19596.csv')

# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

# Check unique values for a supposed unique identifier
if 'player_id' in df.columns:
    print("Unique player IDs count:", df['player_id'].nunique())

# Get a quick statistical summary
print("Statistical summary of numerical features:")
print(df.describe())

# Check data types
print("Data types of each column:")
print(df.dtypes)


Missing values per column:
old_game_id               0
qb_id                     0
time_of_day               0
temp                      0
wind                      0
roof                      0
surface                   0
qb_rush_plays_per_game    0
qb_total_rush_plays       0
dtype: int64
Statistical summary of numerical features:
        old_game_id    qb_id        temp        wind        roof     surface  \
count  2.920000e+02    292.0  292.000000  292.000000  292.000000  292.000000   
mean   2.014623e+09  19596.0   57.345890    7.267123    0.143836    0.633562   
std    4.887512e+06      0.0   16.996873    5.497630    0.351525    0.482658   
min    2.006091e+09  19596.0   13.000000    0.000000    0.000000    0.000000   
25%    2.011071e+09  19596.0   44.000000    3.000000    0.000000    0.000000   
50%    2.015016e+09  19596.0   59.500000    7.000000    0.000000    1.000000   
75%    2.019091e+09  19596.0   70.000000   11.000000    0.000000    1.000000   
max    2.023012e+09  1959

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Load your dataset


# Handle NaN values for 'qb_id' and convert to int
# Assuming we're filling NaNs with -1 as a placeholder (adjust as necessary)
df['qb_id'] = df['qb_id'].fillna(-1).astype(int)

# Convert 'time_of_day' from object to datetime (no extraction of hour unless you need it)
df['time_of_day'] = pd.to_datetime(df['time_of_day'], errors='coerce')

# Assuming 'roof' and 'surface' are categorical but kept as integers if they are already appropriately coded
# If these need to be treated as categorical for modeling and are not just IDs, consider this:
# encoder = OneHotEncoder()
# encoded_features = encoder.fit_transform(df[['roof', 'surface']])
# encoded_df = pd.DataFrame(encoded_features.toarray(), columns=encoder.get_feature_names_out(['roof', 'surface']))
# df = pd.concat([df, encoded_df], axis=1)  # This adds encoded columns to the dataframe
# df.drop(['roof', 'surface'], axis=1, inplace=True)  # This removes original columns

# Now let's print out the updated data types to confirm changes
print(df.dtypes)

# Optionally, save the processed dataframe
df.to_csv('processed_dataset_for_one.csv', index=False)


NameError: name 'df' is not defined

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
data = df.copy()

# Calculate correlation matrix
corr = data[['temp', 'wind', 'roof','time_of_day', 'surface', 'qb_rush_plays_per_game', 'qb_total_rush_plays']].corr()

# Plot the heatmap
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()



In [ ]:
###############################################################
#
#                 For one (dummy)
#
###############################################################

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error

def prepare_data(df, target):
    """Prepare training and prediction datasets."""
    # Assuming the DataFrame is already sorted or sorting by a datetime field if applicable
    # df.sort_values('date_column_name', inplace=True)

    # Extracting the last value as the dummy prediction
    last_value = df[target].iloc[-1]
    return last_value

def predict_with_dummy(df, last_value):
    """Use the dummy model to predict using the last known value."""
    # Create a dummy prediction array with the last known value for the size of the dataset
    dummy_predictions = np.full(shape=(len(df),), fill_value=last_value, dtype='float')
    return dummy_predictions

def evaluate_model(y_true, y_pred):
    """Evaluate the dummy model using MAE."""
    mae = mean_absolute_error(y_true, y_pred)
    return mae

# Use the existing DataFrame 'data'
if __name__ == "__main__":
    # Assume 'data' is your DataFrame already loaded in the environment
    df = data.copy()  # Work with a copy to avoid altering the original data

    # Prepare the data
    target = 'qb_total_rush_plays'  # Specify the target column name
    last_known_value = prepare_data(df, target)

    # Predict using the dummy model
    predictions = predict_with_dummy(df, last_known_value)

    # Evaluate the model using the actual data
    true_labels = df[target]  # Adjust if you have a separate test set
    mae = evaluate_model(true_labels, predictions)
    print("Mean Absolute Error of Dummy Model:", mae)


Mean Absolute Error of Dummy Model: 8.825342465753424


In [ ]:
###############################################################
#
#                 For one (simple decision tree)
#
###############################################################

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming 'data' is your DataFrame
X = data[['temp', 'wind', 'roof', 'surface', 'qb_rush_plays_per_game',]]
y = data['qb_total_rush_plays']  # Target variable

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the RandomForest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
predictions = model.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print("Test RMSE:", rmse)


Test RMSE: 9.898163247322437


In [ ]:
###############################################################
#
#                 For one (adv decision tree)
#
###############################################################


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Load the dataset
df = data.copy() # Changed from data.copy to data.copy()

# Define features and target variable
features = ['temp', 'wind', 'roof', 'surface', 'qb_rush_plays_per_game']
target = 'qb_total_rush_plays'

# Extract features and target from the DataFrame
X = df[features]
y = df[target]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the XGBRegressor model
model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model using mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Since there's no 'player_name', focus on the maximum rush attempt prediction
df_test = df.iloc[y_test.index].copy()
df_test['predicted_rush_attempts'] = y_pred
max_rush_attempt = df_test['predicted_rush_attempts'].max()
print(f"Maximum predicted rush attempts: {max_rush_attempt}")

Mean Absolute Error: 7.913637516862255
Maximum predicted rush attempts: 48.20990753173828


In [ ]:
###############################################################
#
#                 For one (adv decision tree)
#
###############################################################

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the dataset
df = data.copy()

# Assuming 'roof' and 'surface' might be categorical
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(df[['roof', 'surface']])
# Check your scikit-learn version and use the appropriate method
try:
    encoded_df = pd.DataFrame(encoded_features.toarray(), columns=encoder.get_feature_names_out(['roof', 'surface']))
except AttributeError:  # Fallback for older versions of scikit-learn
    encoded_df = pd.DataFrame(encoded_features.toarray(), columns=encoder.get_feature_names(['roof', 'surface']))
df = pd.concat([df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

# Define features and target variable
features = ['temp', 'wind'] + list(encoded_df.columns) + ['qb_rush_plays_per_game']
target = 'qb_total_rush_plays'

# Extract features and target from the DataFrame
X = df[features]
y = df[target]

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the XGBRegressor model
model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
model.fit(X_train, y_train)

# Predict on the test set and evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2}")

# Predictions for next season based on historical averages
next_season_prediction = model.predict(pd.DataFrame([X.mean()], columns=X.columns))
print("Predicted total rush attempts for next season:", next_season_prediction[0])


Mean Absolute Error: 7.913637516862255
Mean Squared Error: 99.59691828190829
R² Score: -0.18545867283275808
Predicted total rush attempts for next season: 38.099083


In [ ]:
###############################################################
#
#                  For one (random forrest)
#
###############################################################

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Load the dataset
df = data.copy()  # Assuming 'data' is pre-loaded DataFrame

# Define features and target variable
features = ['temp', 'wind', 'roof', 'surface', ]
target = 'qb_total_rush_plays'

# Extract features and target from the DataFrame
X = df[features]
y = df[target]

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.05, 0.1, 0.15],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=XGBRegressor(random_state=42), param_grid=param_grid, scoring='neg_mean_absolute_error', cv=3, verbose=1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best estimator found by GridSearch
best_model = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)

# Predict on the test set with the best model
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"\nTest Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2}")

# Feature Importance
feature_importances = best_model.feature_importances_
sorted_idx = np.argsort(feature_importances)[::-1]
plt.figure(figsize=(10, 7))
plt.title("Feature Importances")
plt.bar(range(X.shape[1]), feature_importances[sorted_idx], align='center')
plt.xticks(range(X.shape[1]), np.array(features)[sorted_idx], rotation=90)
plt.show()

# Assuming you need to estimate conditions for next season's games
average_temp = df['temp'].mean()
average_wind = df['wind'].mean()
most_common_roof = int(df['roof'].mode()[0])
most_common_surface = int(df['surface'].mode()[0])
#average_qb_rush_plays_per_game = df['qb_rush_plays_per_game'].mean()

# Create a DataFrame for next season's predicted games based on historical averages
next_season_features = pd.DataFrame({
    'temp': [average_temp],
    'wind': [average_wind],
    'roof': [most_common_roof],
    'surface': [most_common_surface],
    #'qb_rush_plays_per_game': [average_qb_rush_plays_per_game]
})

# Predict next season's total rush attempts using the best model
next_season_prediction = best_model.predict(next_season_features)
print("Predicted total rush attempts for next season:", next_season_prediction[0])


Fitting 3 folds for each of 243 candidates, totalling 729 fits
Best parameters found:  {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.8}

Test Mean Absolute Error: 6.967039851819054
Mean Squared Error: 74.92963204229146
R² Score: 0.10814527508491267


Predicted total rush attempts for next season: 39.89997


In [ ]:
###############################################################
#
#                       For one (linear regression)
#
###############################################################

#Linear regression

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Assuming 'data' is a pre-loaded DataFrame
df = data.copy()  # Make a copy to avoid changing the original data

# Define features and target variable
features = ['temp', 'wind', 'roof', 'surface', 'qb_rush_plays_per_game']
target = 'qb_total_rush_plays'

# Extract features and target from the DataFrame
X = df[features]
y = df[target]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model using mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Optional: Predicting future data if needed
# Prepare new data similar to 'next_season_features' if needed and predict
# Example - predict using mean values from the current dataset
mean_values = df[features].mean().to_frame().T
future_prediction = model.predict(mean_values)
print("Predicted future value:", future_prediction[0])


Mean Absolute Error: 7.291130788895559
Predicted future value: 36.764354191859354


In [ ]:
import pandas as pd

df = pd.read_csv('cleaned_final_filtered_file_all_qbs2_all_seasons_with_per_game_with_touchdowns.csv')

len(df['qb_id'].unique())

292

In [ ]:


# Load the dataset
df = pd.read_csv('cleaned_final_filtered_file_all_qbs2_all_seasons_with_per_game_with_touchdowns.csv')

# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

# Check unique values for a supposed unique identifier
if 'player_id' in df.columns:
    print("Unique player IDs count:", df['player_id'].nunique())

# Get a quick statistical summary
print("Statistical summary of numerical features:")
print(df.describe())

# Check data types
print("Data types of each column:")
print(df.dtypes)


Missing values per column:
old_game_id                 0
qb_id                       0
defteam                     0
time_of_day               666
temp                        0
wind                        0
roof                        0
surface                     0
total_td_per_game           0
pass_td_per_game            0
rush_td_per_game            0
qb_rush_plays_per_game      0
qb_total_rush_plays         0
dtype: int64
Statistical summary of numerical features:
        old_game_id         qb_id          temp          wind          roof  \
count  1.090200e+04  10902.000000  10902.000000  10902.000000  10902.000000   
mean   2.014228e+09  25677.463952     61.286828      5.997340      0.293524   
std    4.969838e+06   7175.683455     15.469712      5.993965      0.455397   
min    2.006091e+09    865.000000     -6.000000      0.000000      0.000000   
25%    2.010100e+09  22803.000000     52.000000      0.000000      0.000000   
50%    2.014111e+09  26158.000000     68.000000      

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Load your dataset


# Handle NaN values for 'qb_id' and convert to int
# Assuming we're filling NaNs with -1 as a placeholder (adjust as necessary)
df['qb_id'] = df['qb_id'].fillna(-1).astype(int)

# Convert 'time_of_day' from object to datetime (no extraction of hour unless you need it)
df['time_of_day'] = pd.to_datetime(df['time_of_day'], errors='coerce')

# Assuming 'roof' and 'surface' are categorical but kept as integers if they are already appropriately coded
# If these need to be treated as categorical for modeling and are not just IDs, consider this:
# encoder = OneHotEncoder()
# encoded_features = encoder.fit_transform(df[['roof', 'surface']])
# encoded_df = pd.DataFrame(encoded_features.toarray(), columns=encoder.get_feature_names_out(['roof', 'surface']))
# df = pd.concat([df, encoded_df], axis=1)  # This adds encoded columns to the dataframe
# df.drop(['roof', 'surface'], axis=1, inplace=True)  # This removes original columns

# Now let's print out the updated data types to confirm changes
print(df.dtypes)

# Optionally, save the processed dataframe
df.to_csv('processed_dataset_for_all.csv', index=False)

filtered_df = df.copy()


old_game_id                             int64
qb_id                                   int64
defteam                                object
time_of_day               datetime64[ns, UTC]
temp                                  float64
wind                                  float64
roof                                    int64
surface                                 int64
total_td_per_game                     float64
pass_td_per_game                      float64
rush_td_per_game                      float64
qb_rush_plays_per_game                float64
qb_total_rush_plays                     int64
dtype: object


In [ ]:
###############################################################
#
#                       For All (dummy)
#
###############################################################

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error

def prepare_data(df, target):
    """Extract the last value from the data for dummy prediction."""
    if len(df) >= 10:  # Assuming at least one past game data is available for prediction
        last_value = df[target].iloc[-1]
        return last_value
    return None

def predict_with_dummy(df, last_value):
    """Generate dummy predictions using the last known value."""
    dummy_predictions = np.full(shape=(len(df),), fill_value=last_value, dtype='float')
    return dummy_predictions

def evaluate_model(y_true, y_pred):
    """Calculate and return the mean absolute error."""
    mae = mean_absolute_error(y_true, y_pred)
    return mae

# Load the dataset
df = filtered_df

# Define the target variable
target = 'qb_rush_plays_per_game'

results = []  # List to store results for each QB

# Iterate over each unique QB
for qb_id in df['qb_id'].unique():
    # Filter data for the current QB
    qb_data = df[df['qb_id'] == qb_id]

    # Prepare the last known value from the QB's data
    last_known_value = prepare_data(qb_data, target)

    if last_known_value is not None:
        # Predict using the dummy model
        predictions = predict_with_dummy(qb_data[:-1], last_known_value)  # Exclude the last game to use as a ground truth

        # Evaluate the model
        mae = evaluate_model(qb_data[target].iloc[1:], predictions)  # Use all except the first game for evaluation

        # Store results
        results.append({
            'QB ID': qb_id,
            'Mean Absolute Error': mae,
            'Predicted Rush Attempts': last_known_value
        })
    else:
        print(f"QB ID {qb_id}: Not enough data for prediction.")

# Convert results to DataFrame for easier analysis
results_df = pd.DataFrame(results)
print(results_df)

# Optionally, save results to CSV
results_df.to_csv('qb_dummy_model_results_per_game.csv', index=False)


QB ID 22005: Not enough data for prediction.
QB ID 1361: Not enough data for prediction.
QB ID 1823: Not enough data for prediction.
QB ID 20405: Not enough data for prediction.
QB ID 22026: Not enough data for prediction.
QB ID 18272: Not enough data for prediction.
QB ID 20305: Not enough data for prediction.
QB ID 21379: Not enough data for prediction.
QB ID 20434: Not enough data for prediction.
QB ID 19623: Not enough data for prediction.
QB ID 23859: Not enough data for prediction.
QB ID 22039: Not enough data for prediction.
QB ID 10455: Not enough data for prediction.
QB ID 24824: Not enough data for prediction.
QB ID 22590: Not enough data for prediction.
QB ID 7576: Not enough data for prediction.
QB ID 4293: Not enough data for prediction.
QB ID 19587: Not enough data for prediction.
QB ID 22744: Not enough data for prediction.
QB ID 25427: Not enough data for prediction.
QB ID 23158: Not enough data for prediction.
QB ID 21277: Not enough data for prediction.
QB ID 26234: N

In [ ]:
###############################################################
#
#                For All (linear regression)
#
###############################################################


import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the dataset
df = filtered_df  # Assuming you already filtered it before

# Define features and target
features = ['temp', 'wind', 'roof', 'surface']
target = 'qb_rush_plays_per_game'

# How many future games to predict
n_games_ahead = 3

results = []

# Iterate over each QB
for qb_id in df['qb_id'].unique():
    qb_data = df[df['qb_id'] == qb_id]

    if len(qb_data) < 10:
        print(f"Not enough data for QB ID {qb_id}")
        continue

    # Split chronologically
    test_size = 0.2
    split_index = int((1 - test_size) * len(qb_data))
    X_train = qb_data[features][:split_index]
    y_train = qb_data[target][:split_index]

    # Train the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Start with the last known game's features
    current_features = qb_data[features].iloc[split_index - 1:split_index].copy().values
    predictions = []

    for _ in range(n_games_ahead):
        # Predict
        # Convert current_features back into a DataFrame with correct column names
        current_features_df = pd.DataFrame(current_features, columns=features)

        # Then make the prediction
        pred = model.predict(current_features_df)[0]
        predictions.append(pred)

        # Update features (simulate slight change in temp and wind)
        current_features[0][0] = np.random.normal(loc=current_features[0][0], scale=2)  # temp
        current_features[0][1] = np.random.normal(loc=current_features[0][1], scale=1)  # wind
        # roof and surface remain constant (assuming no change for now)

    # Optional: evaluate model on test set (last 20%)
    X_test = qb_data[features][split_index:]
    y_test = qb_data[target][split_index:]
    y_pred_test = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred_test)
    mse = mean_squared_error(y_test, y_pred_test)
    r2 = r2_score(y_test, y_pred_test)

    results.append({
        'QB ID': qb_id,
        'MAE': mae,
        'Initial MSE': mse,
        'Initial R² Score': r2,
        'Predicted Next Games': predictions
    })

# Results DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv('qb_results_recursive_predictions_linear_regression_per_game.csv', index=False)
print(results_df)




Not enough data for QB ID 22005
Not enough data for QB ID 1361
Not enough data for QB ID 1823
Not enough data for QB ID 20405
Not enough data for QB ID 22026
Not enough data for QB ID 18272
Not enough data for QB ID 20305
Not enough data for QB ID 21379
Not enough data for QB ID 20434
Not enough data for QB ID 19623
Not enough data for QB ID 23859
Not enough data for QB ID 22039
Not enough data for QB ID 10455
Not enough data for QB ID 24824
Not enough data for QB ID 22590
Not enough data for QB ID 7576
Not enough data for QB ID 4293
Not enough data for QB ID 19587
Not enough data for QB ID 22744
Not enough data for QB ID 25427
Not enough data for QB ID 23158
Not enough data for QB ID 21277
Not enough data for QB ID 26234
Not enough data for QB ID 25582
Not enough data for QB ID 25592
Not enough data for QB ID 22181
Not enough data for QB ID 26296
Not enough data for QB ID 22141
Not enough data for QB ID 25970
Not enough data for QB ID 27131
Not enough data for QB ID 26196
Not enough d

In [ ]:
###############################################################
#
#                  For All (random forrest)
#
###############################################################

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Load the dataset
df = filtered_df

# Define features and target variable
features = ['temp', 'wind', 'roof', 'surface']
target = 'qb_rush_plays_per_game'

results = []

# Iterate over each unique QB
for qb_id in df['qb_id'].unique():
    qb_df = df[df['qb_id'] == qb_id]  # Filter data for the current QB

    if len(qb_df) < 10:  # Ensure there is enough data to split and train
        print(f"Not enough data for QB ID {qb_id}")
        continue

    X = qb_df[features]
    y = qb_df[target]

    # Split data into training and test sets maintaining chronological order
    test_size = 0.2
    split_index = int((1 - test_size) * len(X))
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # Define the parameter grid
    param_grid = {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.05, 0.1, 0.15],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0]
    }

    # Initialize the GridSearchCV object
    grid_search = GridSearchCV(estimator=XGBRegressor(random_state=42),
                               param_grid=param_grid,
                               scoring='neg_mean_absolute_error',
                               cv=3, verbose=1)

    # Fit GridSearchCV
    grid_search.fit(X_train, y_train)

    # Best estimator found by GridSearch
    best_model = grid_search.best_estimator_

    # Predict on the test set with the best model
    y_pred = best_model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Store results
    results.append({
        'QB ID': qb_id,
        'MAE': mae,
        'MSE': mse,
        'R² Score': r2,
        'Predicted Rush Attempts': np.mean(y_pred)  # Use average predictions for simplicity
    })

# Convert results to DataFrame for easier analysis
results_df = pd.DataFrame(results)
results_df.to_csv('qb_results_with_predictions_per_game_random_forrest_1.csv', index=False)
print(results_df)


Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 7

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the dataset
df = filtered_df

# Define features and target variable
features = ['temp', 'wind', 'roof', 'surface']
target = 'qb_rush_plays_per_game'

results = []

# Predicting multiple future games
n_games_ahead = 3  # Define how many games into the future you want to predict

# Iterate over each unique QB
for qb_id in df['qb_id'].unique():
    qb_df = df[df['qb_id'] == qb_id]  # Filter data for the current QB

    if len(qb_df) < 10:  # Ensure there is enough data to split and train
        print(f"Not enough data for QB ID {qb_id}")
        continue

    # Split data into training and test sets maintaining chronological order
    test_size = 0.2
    split_index = int((1 - test_size) * len(qb_df))
    X_train, X_test = qb_df[features][:split_index], qb_df[features][split_index:]
    y_train, y_test = qb_df[target][:split_index], qb_df[target][split_index:]

    # Train the XGBRegressor
    model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, subsample=0.8, colsample_bytree=0.8)
    model.fit(X_train, y_train)

    # Start with the last known features
    current_features = X_test.iloc[-1:].values
    predictions = []
    for _ in range(n_games_ahead):
        next_game_prediction = model.predict(current_features)
        predictions.append(next_game_prediction[0])

        # Update current_features for the next prediction
        # This step needs to be customized based on how you manage dynamic feature changes
        # Example: assuming 'temp' and 'wind' can be forecasted or estimated
        current_features[0][0] = np.random.normal(loc=current_features[0][0], scale=2)  # Generates a random number for temp from a gaussian distribution with the mean of last temp asn an std of 2
        current_features[0][1] = np.random.normal(loc=current_features[0][1], scale=1)  # Generates a random number for wind from a gaussian distribution with the mean of last temp asn an std of 1

    # Calculate metrics for the initial test set to understand model performance before forecasting
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Store results
    results.append({
        'QB ID': qb_id,
        'MAE': mae,
        'Initial MSE': mse,
        'Initial R² Score': r2,
        'Predictions for Next Games': predictions
    })

# Convert results to DataFrame for easier analysis
results_df = pd.DataFrame(results)
results_df.to_csv('qb_results_with_predictions_per_game_random_forrest_2.csv', index=False)
print(results_df)


Not enough data for QB ID 22005
Not enough data for QB ID 1361
Not enough data for QB ID 1823
Not enough data for QB ID 20405
Not enough data for QB ID 22026
Not enough data for QB ID 18272
Not enough data for QB ID 20305
Not enough data for QB ID 21379
Not enough data for QB ID 20434
Not enough data for QB ID 19623
Not enough data for QB ID 23859
Not enough data for QB ID 22039
Not enough data for QB ID 10455
Not enough data for QB ID 24824
Not enough data for QB ID 22590
Not enough data for QB ID 7576
Not enough data for QB ID 4293
Not enough data for QB ID 19587
Not enough data for QB ID 22744
Not enough data for QB ID 25427
Not enough data for QB ID 23158
Not enough data for QB ID 21277
Not enough data for QB ID 26234
Not enough data for QB ID 25582
Not enough data for QB ID 25592
Not enough data for QB ID 22181
Not enough data for QB ID 26296
Not enough data for QB ID 22141
Not enough data for QB ID 25970
Not enough data for QB ID 27131
Not enough data for QB ID 26196
Not enough d

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the dataset
df = filtered_df  # Assuming 'filtered_df' is already loaded and prepared.

# Define features and target variable
features = ['temp', 'wind', 'roof', 'surface']
target = 'qb_rush_plays_per_game'

results = []

# Predicting multiple future games
n_games_ahead = 3  # Define how many games into the future you want to predict

# Iterate over each unique QB
for qb_id in df['qb_id'].unique():
    qb_df = df[df['qb_id'] == qb_id]  # Filter data for the current QB

    if len(qb_df) < 10:  # Ensure there is enough data to split and train
        print(f"Not enough data for QB ID {qb_id}")
        continue

    # Split data into training and test sets maintaining chronological order
    test_size = 0.2
    split_index = int((1 - test_size) * len(qb_df))
    X_train, X_test = qb_df[features][:split_index], qb_df[features][split_index:]
    y_train, y_test = qb_df[target][:split_index], qb_df[target][split_index:]

    # Define the parameter grid
    param_grid = {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.05, 0.1, 0.15],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.7, 0.8, 0.9]
    }

    # Setup GridSearchCV
    grid_search = GridSearchCV(estimator=XGBRegressor(random_state=42),
                               param_grid=param_grid,
                               scoring='neg_mean_squared_error',
                               cv=3, verbose=1)

    # Fit GridSearchCV
    grid_search.fit(X_train, y_train)

    # Best estimator found by GridSearch
    best_model = grid_search.best_estimator_

    # Start with the last known features for recursive forecasting
    current_features = X_test.iloc[-1:].values
    predictions = []
    for _ in range(n_games_ahead):
        next_game_prediction = best_model.predict(current_features)
        predictions.append(next_game_prediction[0])

        # Update current_features for the next prediction
        current_features[0][0] = np.random.normal(loc=current_features[0][0], scale=2)  # Generates a random number for temp from a gaussian distribution with the mean of last temp asn an std of 2
        current_features[0][1] = np.random.normal(loc=current_features[0][1], scale=1)  # Generates a random number for wind from a gaussian distribution with the mean of last temp asn an std of 1

    # Calculate metrics for the initial test set to understand model performance before forecasting
    y_pred = best_model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Store results
    results.append({
        'QB ID': qb_id,
        'MAE': mae,
        'Initial MSE': mse,
        'Initial R² Score': r2,
        'Predictions for Next Games': predictions
    })

# Convert results to DataFrame for easier analysis
results_df = pd.DataFrame(results)
results_df.to_csv('qb_results_with_predictions_per_game_random_forrest_3.csv', index=False)
print(results_df)


Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 729 fits
Fitting 3 folds for each of 243 candidates, totalling 7